In [ ]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from pprint import pprint
import folium
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from scipy import stats
from sklearn.preprocessing import PowerTransformer
from collections import Counter
import json

In [ ]:
class airbnb_city:
    
    def __init__(self, csv):
        
        self.csv = csv
                
        self.df_city = pd.read_csv(self.csv)
        
        print("Instance created!")
        
    def clean_columns(self):
        
        # Take only the relevant columns in the dataframe
        
        l_relevant_columns = ["host_is_superhost","neighbourhood_cleansed","neighbourhood_group_cleansed","property_type","room_type","accommodates","bathrooms_text","beds","price","minimum_nights","maximum_nights","availability_30","availability_365","number_of_reviews","instant_bookable", "amenities", "host_verifications"]

        self.df_city = self.df_city[l_relevant_columns]
        
        self.df_city["bathrooms_text"].replace(np.nan, "?", inplace = True)
        
        # Get numbers out of bathroom_text columns
        
        l_nums = [re.findall(r'\d+',i) for i in self.df_city["bathrooms_text"].values]

        l_nums_completed = []

        for i in l_nums:

            if len(i) > 1:

                l_nums_completed.append('.'.join(i))

            elif len(i) == 0:

                l_nums_completed.append('0')

            else:

                l_nums_completed.append(i[0])

        # Separate categories from bathroom_text
        
        l_category = []

        for i in self.df_city["bathrooms_text"].values:

            if "shared" in i:

                l_category.append("Shared")

            elif "private" in i:

                l_category.append("Private")

            else:

                l_category.append("Unknown")
                
        # Create two different columns replacing bathroom_text
        
        self.df_city.drop("bathrooms_text", axis = 1, inplace = True)
                
        self.df_city["num_of_baths"] = l_nums_completed
        
        self.df_city["bath_category"] = l_category
        
        self.df_city["num_of_baths"] = self.df_city["num_of_baths"].astype("float64")

        # Column["prices"]
        
        self.df_city["price"]  = self.df_city["price"] .apply(lambda x: float(x.strip("$").replace(',', '')) if pd.notnull(x) else x)
        
        self.df_city["amenities"] = [len(i) for i in self.df_city["amenities"]]

        self.df_city["host_verifications"] = [len(i) for i in self.df_city["host_verifications"]]
        
        
        print("Dataframe sucessfully created!")
    
    def label_encoding(self):
        
        self.df_city.dropna(inplace=True)
        
        # Encoding columns with dummies function
        
        def dummies(data, column):
            return pd.get_dummies(data = data[column], drop_first=True)
        
        self.df_city["host_is_superhost"] = dummies(self.df_city, "host_is_superhost")
        self.df_city["instant_bookable"] = dummies(self.df_city, "instant_bookable")
        
        df_room_type = dummies(self.df_city, "room_type")
        df_bath_category = dummies(self.df_city, "bath_category")
        df_bath_category = df_bath_category.rename(columns={'Shared': 'shared_bath', 'Unknown': 'unknoun_bath'})
        
        self.df_city = pd.concat([self.df_city, df_bath_category], axis = 1)

        self.df_city = pd.concat([self.df_city, df_room_type], axis = 1)

        self.df_city.drop("room_type", axis = 1, inplace = True)

        self.df_city.drop("bath_category", axis = 1, inplace = True)
        
        # Encoding categorical columns with labelEncoding function
        
        l_columns_to_labelEncode = ["neighbourhood_cleansed", "property_type", "neighbourhood_group_cleansed"]
        l_columns_encoded = list()

        for i in l_columns_to_labelEncode:

            # Inicializing object LabelEncoder()
            o_labelEncoding = LabelEncoder()

            # Training it with the column data
            o_labelEncoding.fit(self.df_city[i].values)

            # Transform the column
            l_columns_encoded.append(o_labelEncoding.transform(self.df_city[i].values))

        self.df_city["neighbourhood_cleansed"] = l_columns_encoded[0]
        self.df_city["property_type"] = l_columns_encoded[1]
        self.df_city["neighbourhood_group_cleansed"] = l_columns_encoded[2]
        
        print("Dataframe sucessfully encoded!")

        
    def return_df(self):
    
        return self.df_city
    
    def display_df(self):
    
        display(self.df_city)

In [ ]:
madrid = airbnb_city("C:\\Users\\ignci\\OneDrive\\Escritorio\\Curso\\Coisigna\\dsb-p2-ml\\ipynbs\datasets\\Madrid air bnb\\madrid.csv")

In [ ]:
df = madrid.return_df()

In [ ]:
df["price"] = df["price"].apply(lambda x: float(x.strip("$").replace(',', '')) if pd.notnull(x) else x).values

In [ ]:
df = df[df["price"]<700]

## Cleaning property_type

In [ ]:
from collections import Counter
dict_property_types = dict(Counter(df["property_type"]))

dict_property_types[list(dict_property_types.keys())[0]]

list_others = []
for i in dict_property_types.keys():
    
    if dict_property_types[i] < 300:
        
        list_others.append(i)
        
for i in list_others:
    
    df["property_type"].replace(i,"Other", inplace = True)

df["property_type"].value_counts()

### Encoding property_type with the mean values of the price

In [ ]:
# df.groupby("property_type").mean().price

# for i, j in zip(df.groupby("property_type").mean().price.index, df.groupby("property_type").mean().price.values):
#     df["property_type"].replace(i, j, inplace = True)

### Encoding property_type with get dummies

In [ ]:
# df_property_type = dummies(df, "property_type")

# df = pd.concat([df, df_property_type], axis = 1)

# df.drop("property_type", axis = 1, inplace = True)

# df_property_type.columns

### Encoding property_type with label encoder

In [ ]:
# Inicializing object LabelEncoder()
o_labelEncoding = LabelEncoder()

# Training it with the column data
o_labelEncoding.fit(df["property_type"].values)

# Transform the column
property_type = o_labelEncoding.transform(df["property_type"].values)

In [ ]:
df["property_type"] = property_type

# neighbourhood_group_cleansed

## Encoding with label encoder

In [ ]:
# Inicializing object LabelEncoder()
o_labelEncoding = LabelEncoder()

# Training it with the column data
o_labelEncoding.fit(df["neighbourhood_group_cleansed"].values)

# Transform the column
neighbourhood_group_cleansed = o_labelEncoding.transform(df["neighbourhood_group_cleansed"].values)

df["neighbourhood_group_cleansed"] = neighbourhood_group_cleansed

## Encoding with C.P.

In [ ]:
# d_neighbourhood_group_cleansed_cp = {"Centro": 28013, "Arganzuela": 28045, "Retiro": 28007, "Salamanca": 28001, "Chamartín": 28002, "Tetuán": 28020, "Chamberí": 28010, "Fuencarral - El Pardo": 28029, "Moncloa - Aravaca": 28008, "Latina": 28044, "Carabanchel": 28025, "Usera": 28026, "Puente de Vallecas": 28038, "Moratalaz": 28030, "Ciudad Lineal": 28037, "Hortaleza": 28043, "Villaverde": 28021, "Villa de Vallecas": 28031, "Vicálvaro": 28032, "San Blas - Canillejas": 28022, "Barajas": 28042}

# for i, j in d_neighbourhood_group_cleansed_cp.items():
#     df["neighbourhood_group_cleansed"].replace(i, j, inplace=True)

# df["neighbourhood_group_cleansed"].unique()

## Cleaning room_type

In [ ]:
def dummies(data, column):
    return pd.get_dummies(data = data[column])

In [ ]:
df_room_type = dummies(df, "room_type")

df = pd.concat([df, df_room_type], axis = 1)

df.drop("room_type", axis = 1, inplace = True)

df_room_type.columns

## Cleaning amenities

In [ ]:
import json
l_amenities_cleaned = list()
for i in df["amenities"]:
    
    l_amenities_cleaned.append(json.loads(i))

In [ ]:
l_amenities_cleaned

In [ ]:
# More relevant amenities, to see why check EDA _2_from_bedrooms.ipynb


l_amenities_valuables = ['Long term stays allowed','Cooking basics','Dishes and silverware','Essentials','Coffee maker','Hair dryer','Microwave','Refrigerator','Heating','Air conditioning']
                         
                         

In [ ]:
df_amenities_valuables = pd.DataFrame()

for j in l_amenities_valuables:
    
    df_amenities_valuables[j] = [1 if j in i else 0 for i in l_amenities_cleaned]

In [ ]:
df_amenities_valuables

In [ ]:
df_amenities_valuables.columns

In [ ]:
df = pd.concat([df, df_amenities_valuables], axis = 1)

# Test 1: 

## R**2 = 0.43

In [ ]:
# df_columns_chosen = df[["host_id", "neighbourhood_group_cleansed", "latitude", "longitude", "Hotel room", "Private room","Shared room"  ,"accommodates", "price","minimum_nights","availability_365","number_of_reviews","calculated_host_listings_count", "reviews_per_month"]]

# Test 2:

## R**2 = 0.25

In [ ]:
# df_columns_chosen = df[["neighbourhood_group_cleansed","accommodates", "price","minimum_nights","availability_365","number_of_reviews","calculated_host_listings_count", "reviews_per_month"]]

# Test: Adding property_type

## Test 3.1: R**2 = 0.30 (Encoding property_type with the mean values of the price)

In [ ]:
# df_columns_chosen = df[["neighbourhood_group_cleansed","accommodates", "price", "property_type", "minimum_nights","availability_365","number_of_reviews","calculated_host_listings_count", "reviews_per_month"]]

## Test 3.2: R**2 = 0.26 (Encoding property_type with get dummies)

In [ ]:
# # df_columns_chosen = df[["neighbourhood_group_cleansed","accommodates", "price", 'Entire apartment', 'Entire condominium', 'Entire loft', 'Other',
#                          'Private room in apartment', 'Private room in condominium',
#                                'Private room in house', "minimum_nights","availability_365","number_of_reviews","calculated_host_listings_count", "reviews_per_month"]]

##  Test 3.3: R**2 = 0.30 (Encoding property_type with label encoder)

In [ ]:
# df_columns_chosen = df[["neighbourhood_group_cleansed","accommodates", "price", "property_type", "minimum_nights","availability_365","number_of_reviews","calculated_host_listings_count", "reviews_per_month"]]

### It does not seem to be relevant to encode property_type either with label encoding or with the mean values of the price. It gets worst results to do it with get dummies

# Test 4: Adding room_type

## R**2 = 0.29

In [ ]:
# df_columns_chosen = df[["neighbourhood_group_cleansed","accommodates", "price", "property_type",'Entire home/apt', 'Hotel room', 'Private room', 'Shared room', "minimum_nights","availability_365","number_of_reviews","calculated_host_listings_count", "reviews_per_month"]]

### It does not seem to improve with room_type therefor it will be discarded

# Test 5: subtracting calculated_host_listings_count

## R**2 = 0.22

In [ ]:
# df_columns_chosen = df[["neighbourhood_group_cleansed","accommodates", "price", "property_type", "minimum_nights","availability_365","number_of_reviews", "reviews_per_month"]]

### It seem to get worst therefore we add it back

# Test 5: Diferences between encoding with label encode or CP

## Test 5.1: With C.P. encode - R**"2 = 0.24

In [ ]:
# df_columns_chosen = df[["neighbourhood_group_cleansed","accommodates", "price", "property_type", "minimum_nights","availability_365","number_of_reviews","calculated_host_listings_count", "reviews_per_month"]]

## Test 5.2: With label encoder - R**"2 = 0.27

In [ ]:
# df_columns_chosen = df[["neighbourhood_group_cleansed","accommodates", "price", "property_type", "minimum_nights","availability_365","number_of_reviews","calculated_host_listings_count", "reviews_per_month"]]

# Test 6: Adding amenities

## Test 6.1: Addin amenities(Kitchen, TV, Pool, Wifi, Bathtub) -  R**2 = 0.28 Improves just a little

In [ ]:
# df_columns_chosen = df[['Elevator', 'Kitchen', 'Pool', 'Wifi', 'Bathtub',"neighbourhood_group_cleansed","accommodates", "price", "property_type", "minimum_nights","availability_365","number_of_reviews","calculated_host_listings_count", "reviews_per_month"]]

## Test 6.2: Addin the 10 amenities more relevant followin the EDA - R**2 = 0.29

In [ ]:
# df_columns_chosen = df[['Long term stays allowed', 'Cooking basics', 'Dishes and silverware','Essentials', 'Coffee maker', 'Hair dryer', 'Microwave', 'Refrigerator','Heating', 'Air conditioning',"neighbourhood_group_cleansed","accommodates", "price", "property_type", "minimum_nights","availability_365","number_of_reviews","calculated_host_listings_count", "reviews_per_month"]]




# Test 7: Linear regresion

In [ ]:
df_columns_chosen = df[["price", "reviews_per_month"]]


In [ ]:
df_columns_chosen

In [ ]:
df_columns_chosen = df_columns_chosen.dropna()

In [ ]:
df_columns_chosen

In [ ]:
df_columns_chosen.corr()

# Getting rid of outliers using quantiles

In [ ]:
cols = ["price","reviews_per_month"]

for col in cols:
    upper_bound = df_columns_chosen[col].quantile(0.95)
    lower_bound = df_columns_chosen[col].quantile(0.05)
    listings = df_columns_chosen[df_columns_chosen[col] < upper_bound]
    listings = df_columns_chosen[df_columns_chosen[col] > lower_bound]

In [ ]:
# Plot before transformation
stats.probplot(df_columns_chosen["price"], plot=plt)

# Power Transformer
numeric_cols = list(df_columns_chosen._get_numeric_data().columns)
pt = PowerTransformer(method="yeo-johnson")
df_columns_chosen[numeric_cols] = pt.fit_transform(df_columns_chosen[numeric_cols])

In [ ]:
# After transformation
stats.probplot(df_columns_chosen["price"], plot=plt)

# Outliers

In [ ]:
for i in df_columns_chosen.columns:
    
    print(i)
    sns.kdeplot(df_columns_chosen[i])
    plt.show()

## Dividing x & y

In [ ]:
# X = df_columns_chosen.drop("price", axis = 1)
# y = df_columns_chosen[["price"]]


## Scaling data

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import StandardScaler


# scaler_x = StandardScaler()
# scaler_x.fit(X)
# X = scaler_x.transform(X)


# scaler_y = StandardScaler()
# scaler_y.fit(y)
# y = scaler_y.transform(y)

# Feature importances

In [ ]:
df_class = df_columns_chosen["price"].copy()

In [ ]:
df_columns_chosen.drop("price", axis = 1, inplace = True)

In [ ]:
X = np.asarray(df_columns_chosen)
y = np.asarray(df_class)

In [ ]:
X.shape, y.shape

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor

# Build a forest and compute the feature importances
forest = ExtraTreesRegressor(n_estimators = 250,
                              random_state = 0)

forest.fit(X, y)

importances = forest.feature_importances_

std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis = 0)

indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %d (%f): %s" % (f + 1, indices[f], importances[indices[f]], df_columns_chosen.columns[f]))

# Plot the feature importances of the forest
plt.figure()

plt.title("Feature importances")

plt.bar(range(X.shape[1]), importances[indices], color = "r", yerr = std[indices], align = "center")

plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()

# 1. LinearRegression

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=42)

print(f"Train data: {X_train.shape, y_train.shape}")
print(f"Test data: {X_test.shape, y_test.shape}")

In [ ]:
# Regresión algorithm

regresion_lineal = LinearRegression()
regresion_lineal.fit(X_train, y_train)

# Finding coef

print ("weights:", regresion_lineal.coef_)
print ("w_0:", regresion_lineal.intercept_)

## Predicción

In [ ]:
yhat = regresion_lineal.predict(X_test)

for i, j in zip(yhat[:5], y_test[:5]):
    print(f"Predicción:{i} \tValor real:{j}")

## Metrics

In [ ]:
# Sklearn tiene las formulas de algunas métricas en funciones.

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [ ]:
# Relative Absolute Error
RAE = np.sum(np.abs(np.subtract(y_test, yhat))) / np.sum(np.abs(np.subtract(y_test, np.mean(y_test))))

# Relative Square Error
RSE = np.sum(np.square(np.subtract(y_test, yhat))) / np.sum(np.square(np.subtract(y_test, np.mean(y_test))))

# Adjusted R**2
r2_ajustada = 1 - (1 - regresion_lineal.score(X_test, y_test))*(len(y_test) - 1)/(len(y_test) - X_test.shape[1] - 1)

In [ ]:
print(f"MAE:\t {mean_absolute_error(yhat, y_test)}")
print(f"MSE:\t {mean_squared_error(yhat, y_test)}")
print(f"R**2:\t {r2_score(yhat, y_test)}")
print(f"RAE:\t {RAE}")
print(f"RSE:\t {RSE}")
print(f"Adjusted R**2:\t {r2_ajustada}")

In [ ]:
# # Veamos los valores de yhat, y_test y su diferencia

# df_pred = pd.DataFrame()

# df_pred["y_test"] = scaler_y.inverse_transform([y_test]).flatten()
# df_pred["yhat"] = scaler_y.inverse_transform([yhat]).flatten()

# df_pred["diferencia"] = round(abs((df_pred["y_test"] - df_pred["yhat"]) / df_pred["y_test"] * 100), 4)

# df_pred = df_pred.sort_values("diferencia")

# df_pred.head(20)

In [ ]:
# df_pred.tail(20)

In [ ]:
# Distance btw real and predicted values

plt.figure(figsize = (8, 5))

sns.scatterplot(x = y_test.flatten(), y = yhat.flatten(), alpha = 0.5, color = "blue")

plt.xlabel("Valores Reales (y_train)", size = 18)
plt.ylabel("Predicciones (yhat)", size = 18)

plt.show()

# 2. KNeighborsRegressor

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
model = KNeighborsRegressor(n_neighbors = 7)

model.fit(X_train, y_train)

In [ ]:
yhat = model.predict(X_test)

yhat

In [ ]:
print(f"MAE:\t {mean_absolute_error(yhat, y_test)}")
print(f"MSE:\t {mean_squared_error(yhat, y_test)}")
print(f"R**2:\t {r2_score(yhat, y_test)}")
print(f"RAE:\t {RAE}")
print(f"RSE:\t {RSE}")
print(f"Adjusted R**2:\t {r2_ajustada}")

# 3. DecisionTreeRegressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
model = DecisionTreeRegressor(max_depth = 10, max_features = "sqrt", random_state = 42)
model.fit(X_train, y_train)

In [ ]:
yhat = model.predict(X_test)

yhat

In [ ]:
print(f"MAE:\t {mean_absolute_error(yhat, y_test)}")
print(f"MSE:\t {mean_squared_error(yhat, y_test)}")
print(f"R**2:\t {r2_score(yhat, y_test)}")
print(f"RAE:\t {RAE}")
print(f"RSE:\t {RSE}")
print(f"Adjusted R**2:\t {r2_ajustada}")

# 4. RandomForestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
model = RandomForestRegressor(n_estimators = 200, max_depth = 10, random_state = 42, min_samples_split = 20)
model.fit(X_train, y_train)

In [ ]:
yhat = model.predict(X_test)

yhat

In [ ]:
print(f"MAE:\t {mean_absolute_error(yhat, y_test)}")
print(f"MSE:\t {mean_squared_error(yhat, y_test)}")
print(f"R**2:\t {r2_score(yhat, y_test)}")
print(f"RAE:\t {RAE}")
print(f"RSE:\t {RSE}")
print(f"Adjusted R**2:\t {r2_ajustada}")

# 4. AdaBoostRegressor

In [ ]:
from sklearn.ensemble import AdaBoostRegressor

In [ ]:
# %%time

# model = AdaBoostRegressor(base_estimator = RandomForestRegressor(n_estimators      = 200,
#                                                                  max_depth         = 10,
#                                                                  min_samples_split = 20,
#                                                                  n_jobs            = -1),
#                           n_estimators   = 200)

# model.fit(X_train, y_train)

In [ ]:
yhat = model.predict(X_test)

yhat

In [ ]:
print(f"MAE:\t {mean_absolute_error(yhat, y_test)}")
print(f"MSE:\t {mean_squared_error(yhat, y_test)}")
print(f"R**2:\t {r2_score(yhat, y_test)}")
print(f"RAE:\t {RAE}")
print(f"RSE:\t {RSE}")
print(f"Adjusted R**2:\t {r2_ajustada}")

In [ ]:
model.base_estimator_

In [ ]:
plt.plot(model.estimator_errors_, marker = "o", color = "red", linestyle = "")
plt.show()

In [ ]:
importancia = model.feature_importances_

for i in np.argsort(importancia)[::-1]:
    print(i, importancia[i], df_columns_chosen.columns[i])

In [ ]:
df_columns_chosen.columns

In [ ]:
df.columns

In [ ]:
# import pickle
# from sklearn.ensemble import RandomForestClassifier

# model = RandomForestClassifier()
# model.fit(X_train, y_train)

# # Exportar modelos usando pickle con extensión ".sav"
# with open("random_forest_clf.sav", "wb") as file:
#     pickle.dump(model, file)
    
# # Importar modelos usando pickle con extensión ".sav"
# with open("random_forest_clf.sav", "rb") as file:
#     model = pickle.load(file)
    
# # También se puede usar extensión ".pkl"

# Mapa

In [ ]:
# df.columns

In [ ]:
# df['neighbourhood_group_cleansed'].unique()

In [ ]:
# with open("neighbourhoods.geojson", "r") as file:
    
#     geojson = file.read()
    
# pprint(geojson)

In [ ]:
# df_map = df.groupby('neighbourhood_group_cleansed', as_index=False).mean()
# df_map

In [ ]:
# df_map = df_map[df_map["price"]<200]

In [ ]:
# df_map

In [ ]:
# sns.histplot(df_map["price"])

In [ ]:
# world_map = folium.Map(location = [40.18893909782826, -3.938873736090105], zoom_start =12)

# folium.Choropleth(geo_data     = geojson,
#                   data         = df_map,
#                   columns      = ["neighbourhood_group_cleansed", "price"],
#                   key_on       = "feature.properties.neighbourhood_group",
#                   fill_color   = "Blues_r", 
#                   fill_opacity = 0.7, 
#                   line_opacity = 0.2,
#                   legend_name  = "Precio airbnb",
#                   bins         = 253,
#                   highlight    = True).add_to(world_map)

# world_map